In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys

import os
import numpy as np
import pandas as pd
# Custom utils
from utils.simulator.simulator import MCSimulation
# Tf imports
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

2023-04-20 10:38:32.572833: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [63]:
def custom_derivative_model(x, model, multioutput = True, mode = 'centered'):
    h = 1e-1
    # Size x
    x_dim, y_dim, z_dim = x.shape
    print(f'Dimensions: {x_dim}, {y_dim}, {z_dim}')
    # Gradient vector
    gradient = np.zeros((x_dim, y_dim, z_dim))
    for i in range(z_dim):
        for j in range(y_dim):
            # Vector for partial derivative estimation
            offset_tensor = np.zeros((x_dim, y_dim, z_dim))
            offset_tensor[:, j, i] = h
            offset_tensor = tf.convert_to_tensor(offset_tensor,
                                                dtype = tf.float32)
            # Constantes:
            denominator = h
            numerator = tf.math.subtract(
                model(
                    tf.math.add(x, offset_tensor)
                ), model(
                    tf.math.subtract(x, offset_tensor)
                )
            )
            denominator = 2 * h
            gradient[:, j, i] = numerator[:, j, 0] / denominator
    gradient = tf.convert_to_tensor(gradient,
                                        dtype = tf.float32)
    return gradient

In [33]:
input_layer = keras.Input(shape = (10, 2), name='input_nn')
output_layer = layers.GRU(1, 
                dropout = 0.0,
                input_shape = (10, 2),
                return_sequences = True,
                name = 'sequential_layer')(input_layer)
custom_model = keras.Model(
    inputs=[input_layer],
    outputs=[output_layer],
    name = 'test_model'
)

In [94]:
x = tf.convert_to_tensor(
    np.array(range(-30,30)).reshape((3, 10, 2)),
    dtype = tf.float32    
)
# Test 
import time
tf_start = time.time()
xs = tf.Variable(x, trainable = True, name = 'x')
with tf.GradientTape() as tape, tf.GradientTape() as tape_2:
    tape.watch(xs)
    tape_2.watch(xs)
    y = custom_model(xs)
# This represents dV/dX
grads = tape.gradient(y, {
    'x':xs
})
jacobian = tape_2.jacobian(y, {
    'x':xs
})
tf_end = time.time()
print(f'Execution time: {tf_end - tf_start}')

Execution time: 2.752671003341675


In [76]:
custom_start = time.time()
custom_grads = custom_derivative_model(x, custom_model)
custom_end = time.time()
print(f'Execution time custom: {custom_end - custom_start}')

Dimensions: 3, 10, 2
Execution time custom: 0.7661120891571045


In [77]:
custom_grads

<tf.Tensor: shape=(3, 10, 2), dtype=float32, numpy=
array([[[ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00]],

       [[-8.0466270e-06, -8.6426735e-06],
        [ 0.0000000e+00,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00],
        [-3.8743019e-06, -3.8743019e-06],
        [-4.8765540e-03, -4.5439601e-03],
        [-1.7523952e-01, -1.1631742e-01],
        [ 3.6832020e-02,  3.9323568e-02],
        [ 3.5028160e-03,  3.7398934e-03],
        [ 3.3251941e-04,  3.5494566e-04],
        [ 3.1292439e-05,  3.3676624e-05]],

       [[ 2.3841858e-06,  2.6822090e-06],
        [ 2.9802095e-07,  2.9802095e-07],
        [ 0.0000000e

In [81]:
jacobian['x'][0, -1, 0, 0]

<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32)>

In [92]:
jacobian['x'][1, -1, 0, 1]

<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[-1.07120267e-16, -1.14354557e-16],
       [-1.07125673e-16, -1.14360327e-16],
       [ 0.00000000e+00,  0.00000000e+00],
       [-8.91347725e-08, -8.84875675e-08],
       [-1.49133196e-03, -1.39120524e-03],
       [-1.55786052e-01, -1.03299230e-01],
       [ 3.64329405e-02,  3.88953574e-02],
       [ 3.49746319e-03,  3.73366172e-03],
       [ 3.32202093e-04,  3.54637101e-04],
       [ 3.15361540e-05,  3.36659214e-05]], dtype=float32)>